In [3]:
import torch
import torchvision
import pytorchvideo
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
import ffmpeg

import numpy as np
import tensorflow as tf
import cv2
from tqdm import tqdm

from SoccerNet.DataLoader import FrameCV

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
tf.test.gpu_device_name()

2023-03-04 11:50:43.876367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 11:50:45.290383: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-04 11:50:45.290486: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-04 11:50:45.454300: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_nod

'/device:GPU:0'

In [3]:
video_path = '../data/demo/videos/example.mp4'

# ffmpeg change fps

In [4]:
output_path = '../data/demo/videos/example_24fps.mp4'

(
    ffmpeg
    .input(video_path)
    .filter('fps', fps=24, round='up')
    .output(output_path)
    .run()
)

(None, None)

In [5]:
vid = cv2.VideoCapture(output_path)
vid.get(cv2.CAP_PROP_FPS)

24.0

In [6]:
video_path = output_path

# PyTorch vs TF resnet features

In [6]:
tf_feats = np.load('../data/demo/features/example.npy')

## opencv vs pytorchvideo frames subsampling

In [7]:
video_loader = FrameCV(video_path, transform='crop', FPS=2)

Grabbing Video Frames: 100%|██████████| 15321/15321 [00:47<00:00, 321.37frame/s]


In [29]:
video_loader.frames.shape

(1277, 224, 224, 3)

In [43]:
video_loader.frames[:4]

array([[[[ 46,  93,  76],
         [ 46,  93,  76],
         [ 46,  93,  76],
         ...,
         [ 52,  94,  79],
         [ 53,  96,  78],
         [ 53,  96,  78]],

        [[ 46,  93,  76],
         [ 46,  93,  76],
         [ 46,  93,  76],
         ...,
         [ 50,  94,  79],
         [ 51,  95,  79],
         [ 51,  95,  79]],

        [[ 46,  93,  76],
         [ 46,  93,  76],
         [ 46,  93,  76],
         ...,
         [ 50,  95,  79],
         [ 51,  95,  79],
         [ 52,  96,  80]],

        ...,

        [[ 12, 138, 105],
         [ 12, 138, 105],
         [ 13, 138, 105],
         ...,
         [ 12, 144, 107],
         [ 11, 143, 106],
         [ 10, 142, 105]],

        [[ 12, 138, 105],
         [ 12, 138, 105],
         [ 13, 138, 105],
         ...,
         [ 12, 144, 107],
         [ 12, 144, 107],
         [ 11, 143, 106]],

        [[ 12, 138, 105],
         [ 12, 138, 105],
         [ 13, 138, 105],
         ...,
         [ 13, 145, 108],
        

In [44]:
tv_video = EncodedVideo.from_path(video_path)
transform=torchvision.transforms.Compose([
                                    UniformTemporalSubsample(4),
                                    # torchvision.transforms.Lambda(lambda x: x/255.0),
                                    ShortSideScale(224),
                                    lambda x: pytorchvideo.transforms.functional.uniform_crop(x, 224, 1),
                                    # torchvision.transforms.Normalize(
                                    #     mean=[0.485, 0.456, 0.406],
                                    #     std=[0.229, 0.224, 0.225])                                    
                                ])

In [18]:
tv_video.get_clip(0, 10)['video'].shape

torch.Size([3, 250, 720, 1280])

In [ ]:
clip = tv_video.get_clip(0, 2)['video'].to(device)

transform(clip)

In [47]:
clip.permute((1, 2, 3, 0))

tensor([[[[ 29.1071,  33.1071,  35.1071],
          [ 39.4765,  43.4765,  45.4765],
          [ 46.7874,  48.1090,  47.0914],
          ...,
          [ 54.2338,  31.3743,  46.6608],
          [ 67.0000,  24.0000,  46.0000],
          [ 70.0000,  20.0000,  45.0000]],

         [[ 33.7300,  37.7300,  39.7300],
          [ 42.2189,  46.2189,  48.2189],
          [ 52.5573,  53.8789,  52.8613],
          ...,
          [ 54.2338,  31.3743,  46.6608],
          [ 67.0000,  24.0000,  46.0000],
          [ 70.0000,  20.0000,  45.0000]],

         [[ 54.0216,  58.0216,  60.0216],
          [ 54.6071,  58.6071,  60.6071],
          [ 58.0648,  59.3864,  58.3688],
          ...,
          [ 46.2121,  34.4282,  51.7322],
          [ 53.6429,  30.6786,  50.0000],
          [ 57.2002,  24.5573,  47.4145]],

         ...,

         [[102.0000, 135.0000,  13.0000],
          [102.0000, 135.0000,  13.0000],
          [103.0715, 136.0715,  13.3931],
          ...,
          [103.6608, 140.6608,   8.66

## cv2

In [ ]:
cv_vid = cv2.VideoCapture(video_path)

frame_count = int(cv_vid.get(cv2.CAP_PROP_FRAME_COUNT))

frames = []
for frame_idx in tqdm(list(range(frame_count))):
    ret, frame = cv_vid.read()
    
    if not ret:
        break
    
    frames.append(frame)
    
frames = np.array(frames)

In [51]:
pt_frames = torch.from_numpy(video_loader.frames)

In [57]:
frames_np = video_loader.frames.astype('float')
mean = [103.939, 116.779, 123.68]

frames_np = frames_np[..., ::-1]

frames_np[..., 0] -= mean[0]
frames_np[..., 1] -= mean[1]
frames_np[..., 2] -= mean[2]

In [60]:
tf_resnet = tf.keras.applications.ResNet152()
# tf_resnet = tf.keras.models.Model(tf_resnet.input, 
#                                   outputs=[tf_resnet.get_layer("avg_pool").output])

In [63]:
frames_np[0]

array([[[ -27.939,  -23.779,  -77.68 ],
        [ -27.939,  -23.779,  -77.68 ],
        [ -27.939,  -23.779,  -77.68 ],
        ...,
        [ -24.939,  -22.779,  -71.68 ],
        [ -25.939,  -20.779,  -70.68 ],
        [ -25.939,  -20.779,  -70.68 ]],

       [[ -27.939,  -23.779,  -77.68 ],
        [ -27.939,  -23.779,  -77.68 ],
        [ -27.939,  -23.779,  -77.68 ],
        ...,
        [ -24.939,  -22.779,  -73.68 ],
        [ -24.939,  -21.779,  -72.68 ],
        [ -24.939,  -21.779,  -72.68 ]],

       [[ -27.939,  -23.779,  -77.68 ],
        [ -27.939,  -23.779,  -77.68 ],
        [ -27.939,  -23.779,  -77.68 ],
        ...,
        [ -24.939,  -21.779,  -73.68 ],
        [ -24.939,  -21.779,  -72.68 ],
        [ -23.939,  -20.779,  -71.68 ]],

       ...,

       [[   1.061,   21.221, -111.68 ],
        [   1.061,   21.221, -111.68 ],
        [   1.061,   21.221, -110.68 ],
        ...,
        [   3.061,   27.221, -111.68 ],
        [   2.061,   26.221, -112.68 ],
        [

array([[[[ -27.939003,  -23.779   ,  -77.68    ],
         [ -27.939003,  -23.779   ,  -77.68    ],
         [ -27.939003,  -23.779   ,  -77.68    ],
         ...,
         [ -24.939003,  -22.779   ,  -71.68    ],
         [ -25.939003,  -20.779   ,  -70.68    ],
         [ -25.939003,  -20.779   ,  -70.68    ]],

        [[ -27.939003,  -23.779   ,  -77.68    ],
         [ -27.939003,  -23.779   ,  -77.68    ],
         [ -27.939003,  -23.779   ,  -77.68    ],
         ...,
         [ -24.939003,  -22.779   ,  -73.68    ],
         [ -24.939003,  -21.779   ,  -72.68    ],
         [ -24.939003,  -21.779   ,  -72.68    ]],

        [[ -27.939003,  -23.779   ,  -77.68    ],
         [ -27.939003,  -23.779   ,  -77.68    ],
         [ -27.939003,  -23.779   ,  -77.68    ],
         ...,
         [ -24.939003,  -21.779   ,  -73.68    ],
         [ -24.939003,  -21.779   ,  -72.68    ],
         [ -23.939003,  -20.779   ,  -71.68    ]],

        ...,

        [[   1.060997,   21.221   , -1

In [81]:
tf_resnet(tf.keras.applications.resnet.preprocess_input(video_loader.frames[:1]))

<tf.Tensor: shape=(1, 1000), dtype=float32, numpy=
array([[4.45298447e-08, 2.27783925e-08, 3.76046069e-06, 3.30069884e-06,
        8.68861389e-06, 1.60342225e-07, 3.12008339e-08, 2.07283648e-07,
        1.05339371e-09, 3.71193174e-08, 2.74385126e-10, 5.52281314e-08,
        6.94916014e-10, 7.14606013e-11, 2.90764235e-09, 5.38680045e-10,
        3.93495547e-09, 3.20760707e-09, 1.18576207e-08, 1.49675966e-10,
        1.05092735e-09, 2.67806968e-08, 8.74041532e-08, 5.01700370e-09,
        9.95543292e-09, 4.63171146e-09, 4.05271594e-09, 2.55658139e-09,
        2.68760170e-09, 5.06002840e-09, 2.01169339e-08, 6.46503338e-07,
        7.00996523e-08, 4.76295014e-07, 1.81883280e-07, 1.09880671e-09,
        4.44591493e-08, 1.08031140e-09, 1.85065208e-08, 4.85177054e-09,
        1.11725038e-08, 3.43590684e-10, 8.05179312e-10, 6.26113694e-09,
        3.87456289e-09, 8.14490786e-09, 1.61220726e-08, 3.95799660e-09,
        6.13784623e-09, 7.33061256e-08, 5.31358850e-08, 1.93066398e-08,
        8.456

In [84]:
pt_resnet = torchvision.models.resnet152(weights=torchvision.models.ResNet152_Weights.DEFAULT)

In [85]:
pt_resnet = pt_resnet.to(device)
pt_resnet = pt_resnet.eval()

In [86]:
pt_frames_preproc = tf.keras.applications.imagenet_utils.preprocess_input(
    video_loader.frames[:1], 'channels_last', 'torch')

In [87]:
pt_frames_preproc = torch.from_numpy(pt_frames_preproc.transpose(0, 3, 1, 2)).to(device)

In [90]:
torch.nn.functional.softmax(pt_resnet(pt_frames_preproc)[0], dim=0)

tensor([1.1136e-04, 1.9444e-04, 2.2922e-04, 2.5689e-04, 2.0713e-04, 2.5621e-04,
        2.5322e-04, 2.0002e-04, 2.0231e-04, 2.7255e-04, 1.6758e-04, 2.5340e-04,
        1.6995e-04, 1.4325e-04, 2.0044e-04, 2.0814e-04, 1.2148e-04, 2.4466e-04,
        3.1779e-04, 1.4510e-04, 1.6603e-04, 3.0966e-04, 1.9985e-04, 2.3630e-04,
        1.7541e-04, 1.7750e-04, 1.4788e-04, 2.3731e-04, 1.3476e-04, 2.1347e-04,
        1.2855e-04, 3.2765e-04, 1.8858e-04, 1.4270e-04, 1.8817e-04, 1.5691e-04,
        1.6206e-04, 1.4768e-04, 2.4184e-04, 1.3220e-04, 2.2674e-04, 1.6198e-04,
        1.3318e-04, 1.2648e-04, 1.6317e-04, 1.3005e-04, 1.6610e-04, 1.1419e-04,
        1.6395e-04, 1.5466e-04, 1.3068e-04, 2.0125e-04, 1.4480e-04, 2.7031e-04,
        1.8398e-04, 1.8766e-04, 1.6940e-04, 1.3921e-04, 1.6086e-04, 1.0713e-04,
        3.4246e-04, 1.2375e-04, 2.4554e-04, 1.8021e-04, 1.3887e-04, 2.1494e-04,
        2.0644e-04, 1.8613e-04, 1.5357e-04, 1.7492e-04, 2.3001e-04, 1.5280e-04,
        1.5843e-04, 1.5681e-04, 2.2785e-

In [29]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

pt_frames = torch.stack([transform(frame) for frame in video_loader.frames])

In [33]:
pt_frames.shape

torch.Size([1277, 3, 224, 224])

In [42]:
pt_frames.permute((0, 2, 3, 1))

tensor([[[[-1.3302, -0.4076, -0.4798],
          [-1.3302, -0.4076, -0.4798],
          [-1.3302, -0.4076, -0.4798],
          ...,
          [-1.2274, -0.3901, -0.4275],
          [-1.2103, -0.3550, -0.4450],
          [-1.2103, -0.3550, -0.4450]],

         [[-1.3302, -0.4076, -0.4798],
          [-1.3302, -0.4076, -0.4798],
          [-1.3302, -0.4076, -0.4798],
          ...,
          [-1.2617, -0.3901, -0.4275],
          [-1.2445, -0.3725, -0.4275],
          [-1.2445, -0.3725, -0.4275]],

         [[-1.3302, -0.4076, -0.4798],
          [-1.3302, -0.4076, -0.4798],
          [-1.3302, -0.4076, -0.4798],
          ...,
          [-1.2617, -0.3725, -0.4275],
          [-1.2445, -0.3725, -0.4275],
          [-1.2274, -0.3550, -0.4101]],

         ...,

         [[-1.9124,  0.3803,  0.0256],
          [-1.9124,  0.3803,  0.0256],
          [-1.8953,  0.3803,  0.0256],
          ...,
          [-1.9124,  0.4853,  0.0605],
          [-1.9295,  0.4678,  0.0431],
          [-1.9467,  0

In [31]:
tf_frames = tf.keras.applications.resnet.preprocess_input(video_loader.frames)

In [43]:
tf_frames

array([[[[-2.7939003e+01, -2.3778999e+01, -7.7680000e+01],
         [-2.7939003e+01, -2.3778999e+01, -7.7680000e+01],
         [-2.7939003e+01, -2.3778999e+01, -7.7680000e+01],
         ...,
         [-2.4939003e+01, -2.2778999e+01, -7.1680000e+01],
         [-2.5939003e+01, -2.0778999e+01, -7.0680000e+01],
         [-2.5939003e+01, -2.0778999e+01, -7.0680000e+01]],

        [[-2.7939003e+01, -2.3778999e+01, -7.7680000e+01],
         [-2.7939003e+01, -2.3778999e+01, -7.7680000e+01],
         [-2.7939003e+01, -2.3778999e+01, -7.7680000e+01],
         ...,
         [-2.4939003e+01, -2.2778999e+01, -7.3680000e+01],
         [-2.4939003e+01, -2.1778999e+01, -7.2680000e+01],
         [-2.4939003e+01, -2.1778999e+01, -7.2680000e+01]],

        [[-2.7939003e+01, -2.3778999e+01, -7.7680000e+01],
         [-2.7939003e+01, -2.3778999e+01, -7.7680000e+01],
         [-2.7939003e+01, -2.3778999e+01, -7.7680000e+01],
         ...,
         [-2.4939003e+01, -2.1778999e+01, -7.3680000e+01],
         [